In [2]:
import pandas as pd
import os
import numpy as np

print('OK')

OK


In [5]:
full_normal = None
count = 0

for dirname, _, filenames in os.walk('../../Dataset/CTU-13/csv/0'):
    for filename in filenames:
        data = pd.read_csv(os.path.join(dirname, filename))
        full_normal = pd.concat([full_normal, data])
        count+=1
        print(os.path.join(dirname, filename))

print(count)

../../Dataset/CTU-13/csv/0\2017-04-18_win-normal (1).pcap_Flow.csv
../../Dataset/CTU-13/csv/0\2017-04-19_win-normal (1).pcap_Flow.csv
../../Dataset/CTU-13/csv/0\2017-04-25_win-normal.pcap_Flow.csv
../../Dataset/CTU-13/csv/0\2017-05-01_normal (1).pcap_Flow.csv
../../Dataset/CTU-13/csv/0\2017-05-01_normal.pcap_Flow.csv
../../Dataset/CTU-13/csv/0\2017-05-02_normal.pcap_Flow.csv
../../Dataset/CTU-13/csv/0\2017_04_30-normal.pcap_Flow.csv
../../Dataset/CTU-13/csv/0\capture-new-Windows-7-Full.pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017-04-18_win-normal (1).pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017-04-19_win-normal (1).pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017-04-25_win-normal.pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017-05-01_normal (1).pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017-05-01_normal.pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017-05-02_normal.pcap_Flow.csv
../Dataset/CTU-13/csv/0\2017_04_30-normal.pcap_Flow.csv
../Dataset/CTU-13/csv/0\capture-new-Windows-7-Full.pcap_Flow.csv
8


In [6]:
full_malware = None
count = 0

for dirname, _, filenames in os.walk('../../Dataset/CTU-13/csv/1'):
    for filename in filenames:
        data = pd.read_csv(os.path.join(dirname, filename))
        full_malware = pd.concat([full_malware, data])
        count+=1
        print(os.path.join(dirname, filename))

print(count)

../../Dataset/CTU-13/csv/1\botnet-capture-20110810-neris.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110811-neris.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110812-rbot.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110815-fast-flux.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110815-rbot-dos.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110816-donbot.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110816-qvod.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110816-sogou.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110817-bot.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110818-bot-2.pcap_Flow.csv
../../Dataset/CTU-13/csv/1\botnet-capture-20110819-bot.pcap_Flow.csv
../Dataset/CTU-13/csv/1\botnet-capture-20110810-neris.pcap_Flow.csv
../Dataset/CTU-13/csv/1\botnet-capture-20110811-neris.pcap_Flow.csv
../Dataset/CTU-13/csv/1\botnet-capture-20110812-rbot.pcap_Flow.csv
../Dataset/CTU

In [6]:
full_malware.to_csv(path_or_buf = '../../Dataset/CTU-13/malware.csv', index=False)
full_normal.to_csv(path_or_buf = '../../Dataset/CTU-13/normal.csv', index=False)

In [7]:
full_malware.shape

(259503, 84)

In [8]:
full_normal.shape

(340780, 84)

In [10]:
a = full_malware.loc[(full_malware['Src Port'] == 443) | (full_malware['Dst Port'] == 443)]
b = full_normal.loc[(full_normal['Src Port'] == 443) | (full_normal['Dst Port'] == 443)]

In [11]:
a.shape

(18917, 84)

In [12]:
b.shape

(109455, 84)

In [89]:
malware = full_malware.loc[(full_malware['Src Port'] == 443) | (full_malware['Dst Port'] == 443)]
normal = full_normal.loc[(full_normal['Src Port'] == 443) | (full_normal['Dst Port'] == 443)]
malware.drop(['Label'], axis = 1)
malware['Label'] = 1
normal.drop(['Label'], axis = 1)
normal['Label'] = 0
df = pd.concat([malware, normal])
df = df.sample(frac=1, replace=False, random_state=27)
df.head(10)

C:\Users\admin\AppData\Local\Temp\ipykernel_7924\2331738655.py:4: SettingWithCopyWarning: 
C:\Users\admin\AppData\Local\Temp\ipykernel_12420\1866714354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malware['Label'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_7924\2331738655.py:6: SettingWithCopyWarning: 
  a['Label'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_12420\1866714354.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal['Label'] = 0


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
113896,147.32.84.192-46.4.36.120-1512-443-6,147.32.84.192,1512,46.4.36.120,443,6,17/08/2011 08:38:41 PM,1032003,5,4,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
4494,10.0.2.15-104.244.42.195-50684-443-6,104.244.42.195,443,10.0.2.15,50684,6,01/01/1970 09:22:14 AM,22363,2,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0
53723,10.0.2.15-54.80.0.149-59959-443-6,54.80.0.149,443,10.0.2.15,59959,6,01/01/1970 08:48:10 AM,143746,2,2,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0
65904,2.22.109.92-10.0.2.15-443-56891-6,10.0.2.15,56891,2.22.109.92,443,6,01/01/1970 11:13:14 AM,117743736,93,271,...,0,1.542386e+06,3.281592e+06,10952308.0,664.0,9.161106e+06,1.469274e+06,10081852.0,5945825.0,0
29738,172.217.22.70-10.0.2.15-443-63651-6,172.217.22.70,443,10.0.2.15,63651,6,01/01/1970 10:16:12 AM,31663,2,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0
79801,10.0.2.15-69.172.216.55-54123-443-6,10.0.2.15,54123,69.172.216.55,443,6,01/01/1970 11:05:20 AM,2582753,6,7,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0
5244,192.28.146.120-10.0.2.15-443-57194-6,10.0.2.15,57194,192.28.146.120,443,6,01/01/1970 08:30:56 AM,31002,2,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0
18810,147.32.84.165-46.4.36.120-4390-443-6,147.32.84.165,4390,46.4.36.120,443,6,11/08/2011 05:07:01 PM,14775294,14,16,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
16309,147.32.84.207-65.55.196.251-3938-443-6,147.32.84.207,3938,65.55.196.251,443,6,17/08/2011 08:42:59 PM,100346,0,3,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
16550,185.29.135.227-10.0.2.15-443-52497-6,10.0.2.15,52497,185.29.135.227,443,6,01/01/1970 09:21:27 AM,110248097,11,13,...,0,1.831273e+03,4.123128e+02,2834.0,1198.0,1.002052e+07,1.236552e+04,10049835.0,10007985.0,0


In [24]:
df_cp = df.copy()

In [25]:
def clean_df(df):
    # Remove the space before each feature names
    df.columns = df.columns.str.strip()
    print('dataset shape', df.shape)

    # This set of feature should have >= 0 values
    num = df._get_numeric_data()
    num[num < 0] = 0

    zero_variance_cols = []
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(zero_variance_cols, axis = 1, inplace = True)
    print('zero variance columns', zero_variance_cols, 'dropped')
    print('shape after removing zero variance columns:', df.shape)

    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    print(df.isna().any(axis = 1).sum(), 'rows dropped')
    df.dropna(inplace = True)
    print('shape after removing nan:', df.shape)

    # Drop duplicate rows
    df.drop_duplicates(inplace = True)
    print('shape after dropping duplicates:', df.shape)

    column_pairs = [(i, j) for i, j in combinations(df, 2) if df[i].equals(df[j])]
    ide_cols = []
    for column_pair in column_pairs:
        ide_cols.append(column_pair[1])
    df.drop(ide_cols, axis = 1, inplace = True)
    print('columns which have identical values', column_pairs, 'dropped')
    print('shape after removing identical value columns:', df.shape)
    return df
df_cp = clean_df(df_cp)

dataset shape (128372, 84)
zero variance columns ['Fwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Init Fwd Win Byts', 'Fwd Seg Size Min'] dropped
shape after removing zero variance columns: (128372, 70)
29 rows dropped
shape after removing nan: (128343, 70)
shape after dropping duplicates: (128343, 70)
columns which have identical values [('Tot Fwd Pkts', 'Subflow Fwd Pkts'), ('Tot Bwd Pkts', 'Subflow Bwd Pkts'), ('Bwd PSH Flags', 'PSH Flag Cnt')] dropped
shape after removing identical value columns: (128343, 67)


In [26]:
df_cp.Label.value_counts()

Label
0    109426
1     18917
Name: count, dtype: int64

In [27]:
drop_columns = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp']
df_cp = df_cp.drop(drop_columns, axis=1)

In [28]:
features = ['Flow Duration', 'Flow IAT Max', 'Flow Pkts/s', 'Flow IAT Mean', 'Fwd Pkts/s', 'Bwd Pkts/s']
X = pd.get_dummies(df_cp[features])
# X = df_test.drop(['Label'], axis=1)
y = df_cp.Label

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y, random_state = 0, test_size = 0.3)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import svm

In [32]:
model = RandomForestClassifier(random_state=27)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=27)

In [33]:
from sklearn import metrics

preds_val = model.predict(X_test)
accuracy = accuracy_score(y_test, preds_val)
precision = precision_score(y_test, preds_val)
recall = recall_score(y_test, preds_val)
roc_auc = roc_auc_score(y_test, preds_val)

print(accuracy)
print(precision)
print(recall)
print(roc_auc)

0.9954808716203932
0.9923035618399857
0.9769162995594713
0.9878032210603498


In [34]:
features = ['Flow Duration', 'Flow IAT Max', 'Flow Pkts/s', 'Flow IAT Mean', 'Fwd Pkts/s', 'Bwd Pkts/s']
X = pd.get_dummies(df_cp[features])
# X = df_test.drop(['Label'], axis=1)
y = df_cp.Label

In [39]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y, random_state = 0, test_size = 0.7)
model2 = xgb.XGBClassifier(n_estimators=1000, max_depth=3, gamma=0.1, min_child_weight=5)
model2.fit(X_train, y_train)
preds_val = model2.predict(X_test)
classification = metrics.classification_report(y_test, preds_val)
confusion_matrix = metrics.confusion_matrix(y_test, preds_val)
print(accuracy_score(y_test, preds_val))
print("Confusion matrix:" "\n", confusion_matrix)
print("Classification report:" "\n", classification) 

0.9938892042608608
Confusion matrix:
 [[76493   106]
 [  443 12799]]
Classification report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     76599
           1       0.99      0.97      0.98     13242

    accuracy                           0.99     89841
   macro avg       0.99      0.98      0.99     89841
weighted avg       0.99      0.99      0.99     89841



In [40]:
preds_val = model2.predict(X_test)

accuracy = accuracy_score(y_test, preds_val)
precision = precision_score(y_test, preds_val)
recall = recall_score(y_test, preds_val)
roc_auc = roc_auc_score(y_test, preds_val)

print(accuracy)
print(precision)
print(recall)
print(roc_auc)

0.9938892042608608
0.9917861294072066
0.9665458389971303
0.9825810044605099


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y, random_state = 0, test_size = 0.7)

model3 = svm.SVC(kernel='rbf', C=110, gamma=0.1)

model3.fit(X_train, y_train)
preds_val = model2.predict(X_test)
classification = metrics.classification_report(y_test, preds_val)
confusion_matrix = metrics.confusion_matrix(y_test, preds_val)
print(accuracy_score(y_test, preds_val))
print("Confusion matrix:" "\n", confusion_matrix)
print("Classification report:" "\n", classification) 

0.9938892042608608
Confusion matrix:
 [[76493   106]
 [  443 12799]]
Classification report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     76599
           1       0.99      0.97      0.98     13242

    accuracy                           0.99     89841
   macro avg       0.99      0.98      0.99     89841
weighted avg       0.99      0.99      0.99     89841



In [38]:
accuracy = accuracy_score(y_test, preds_val)
precision = precision_score(y_test, preds_val)
recall = recall_score(y_test, preds_val)
roc_auc = roc_auc_score(y_test, preds_val)

print(accuracy)
print(precision)
print(recall)
print(roc_auc)

0.9938892042608608
0.9917861294072066
0.9665458389971303
0.9825810044605099
